## Retriever as a tool
A retriever tool is used to retrieve relevant data from a vector database based on the user input and sent to an LLM as context for the user input. This is done when the source data is large and cannot be send as part of the user input in the prompt. In the below example of retriever tool, data from a website https://www.dasa.org is stored in a vector database, and the retriever tool is used to retrieve relevant information from the vector database to be passed to the LLM as context.

In order to provide the tools to the AI Agent, we first need to create the retriever and then import the tool.

In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.dasa.org")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
docs = loader.load()
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
from langchain_community.vectorstores import FAISS
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

Let’s now import the create_retriever_tool package. We import it as below:

In [4]:
from langchain.tools.retriever import create_retriever_tool

Next, we create the tool by giving it a name and description.

In [5]:
retriever_tool = create_retriever_tool (retriever, "DASA_search", "Search for information about DASA. For any questions about DASA, you must use this tool")

In [6]:
import os 

# Retrieve the Tavily API key from the environment
tavily_api_key = os.getenv("TAVILY_API_KEY")

# Print the value to verify
if tavily_api_key:
    print(f"Tavily API Key found: {tavily_api_key}")
else:
    print("Tavily API Key not found. Please ensure it is set correctly.")


Tavily API Key found: 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


In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults
import os 
search = TavilySearchResults()

In [8]:
tools = [retriever_tool, search]

Next, we will get a predefined prompt from a prompt hub that will be useful to get the llm to act as an AI Agent.

In [9]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt

/Users/aymentiouiri/opt/anaconda3/envs/agents/lib/python3.9/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [11]:
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [19]:
agent = create_openai_functions_agent(llm,tools,prompt)
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [20]:
agent_executor.invoke({"input":"What is the mission of DASA"})



> Entering new AgentExecutor chain...

Invoking: `DASA_search` with `{'query': 'mission of DASA'}`


DASA





















































































AboutThe Enterprise Transformation JourneyWhy DASADASA DevOps PrinciplesProductsGuidance ProductsDASA DevAIOps Value BoxComing SoonDASA Platform Engineering Value BoxDASA Product Management Value BoxDASA Portfolio Management Value BoxDASA Experience Management Value BoxHPDO Getting Started Value BoxDASA Competency FrameworkDigital Readiness AssessmentTalent ProductsCertification ProgramsDASA DevAlOpsDASA Platform EngineeringDASA Experience ManagementDASA DevOps Leader™DASA DevOps Coach™DASA Product ManagementComing soonDASA DevOps Product Owner™DASA Portfolio ManagementDASA DevOps Fundamentals™DASA DevOps Enable and Scale™DASA DevOps Specify and Verify™Role Based JourneysPartnersTransformation PartnersTraining ProvidersMembershipEnterprisesLearnersFor IndividualsFor TeamsTraining CalendarDevOps T

{'input': 'What is the mission of DASA',
 'output': 'The mission of DASA (DevOps Agile Skills Association) is to facilitate the journey towards flow, business agility, and value maximization for organizations. DASA aims to help organizations prioritize customer-focused innovation, foster a culture of agility and teamwork, and enhance flow, agility, and value. DASA emphasizes six key principles crucial for effective adoption and transition to a DevOps-centric approach, focusing on experiences, ideas, and cultural shifts to become a high-performance digital organization. DASA offers a portfolio of talent and guidance products to drive enterprise transformation success and provides a collaborative community for leaders and professionals to share ideas and adopt best practices.'}

In [21]:
agent_executor.invoke({"input":"What is weather in Kuala Lumpur?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Kuala Lumpur'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Kuala Lumpur', 'region': 'Kuala Lumpur', 'country': 'Malaysia', 'lat': 3.1667, 'lon': 101.7, 'tz_id': 'Asia/Kuala_Lumpur', 'localtime_epoch': 1728399493, 'localtime': '2024-10-08 22:58'}, 'current': {'last_updated_epoch': 1728398700, 'last_updated': '2024-10-08 22:45', 'temp_c': 27.3, 'temp_f': 81.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 94, 'wind_dir': 'E', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 84, 'cloud': 25, 'feelslike_c': 30.8, 'feelslike_f': 87.5, 'windchill_c': 25.9, 'windchill_f': 78.6, 'heatindex_c': 28.3, 'heatindex_f': 83.0, 'dewpoint_c': 22.1, 'dewpoint_f': 71.8, 'vis_km': 10.0, 'vis_mil

{'input': 'What is weather in Kuala Lumpur?',
 'output': 'The current weather in Kuala Lumpur is partly cloudy with a temperature of 27.3°C (81.1°F). The humidity is at 84% and there is a light wind blowing from the east at 3.6 km/h.'}